In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
# Libraries 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import os, sys
from keras.models import Model
import csv
from keras.utils import to_categorical

def load_image(root_path, height, width, classes):
    label_map = {'BacterialPneumonia': 1,
                 'COVID-19': 0,
                 'Normal': 2,
                 'ViralPneumonia': 1}
    data, labels = [], []
    dirs = os.listdir(root_path)
    
    for d in dirs:
        print(d)
        path = os.path.join(root_path, d)
        img_count = os.listdir(path)
        for img in img_count:
            try: 
                img_path = os.path.join(path, img)
                image = cv2.imread(img_path)
                image_from_array = Image.fromarray(image, 'RGB')
                size_image = image_from_array.resize((width, height))
                data.append(np.array(size_image))
                labels.append(label_map[d])
            except AttributeError:
                print(" ")
    
    Cells = np.array(data)
    labels = np.array(labels)
    
    s = np.arange(Cells.shape[0])
    np.random.seed(classes)
    np.random.shuffle(s)
    Cells = Cells[s]
    labels = labels[s]
    
    X = Cells.astype('float32')/255
    y = to_categorical(labels, classes)
    return X, y


def data_processing(data_path, height, width, classes):
    train_path = os.path.join(data_path, 'NonAugmentedTrain')
    val_path = os.path.join(data_path, 'ValData')
    X_train, y_train = load_image(train_path, height, width, classes)
    X_val, y_val = load_image(val_path, height, width, classes)
    return X_train, y_train, X_val, y_val

Using TensorFlow backend.


In [ ]:
from __future__ import print_function
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, Input, GlobalAveragePooling2D, AveragePooling2D
from keras import backend as bk
from keras import optimizers
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers import Activation
from keras import initializers, regularizers
import tensorflow as tf
from keras.utils.generic_utils import get_custom_objects
from keras.models import load_model

class Piecewise5(Activation):
    def __init__(self, activation, **kwargs):
        super(Piecewise5, self).__init__(activation, **kwargs)
        self.__name__ = 'Piecewise5'

def get_flops(model):
    run_meta = tf.RunMetadata()
    opts = tf.profiler.ProfileOptionBuilder.float_operation()

    # We use the Keras session graph in the call to the profiler.
    flops = tf.profiler.profile(graph=bk.get_session().graph,
                                run_meta=run_meta, cmd='op', options=opts)

    return flops.total_float_ops


def piecewise5(X):
    return bk.switch(X < -0.6, (0.01 * X ),
                     bk.switch(X < -0.2, (0.2 * X ),
                               bk.switch(X < 0.2, (1 * X ),
                                         bk.switch(X < 0.6, (1.5 * X ),
                                                   bk.switch(X < 5, (3 * X ), (3 * X )))))) 


def custom_network(height, width, channels, classes, pre_trained=''):
    weight_decay = 0.0001
    input_img = Input(shape=(height, width, channels))
    if pre_trained != '':
        base_model = load_model(pre_trained)
        num_layers = len(base_model.layers)
        base_output = base_model.get_layer(index=num_layers-2).output
        out = Dense(classes, activation='softmax')(base_output)
        model = Model(inputs=base_model.input, outputs=out)
    else:
        conv_1 = Conv2D(64, (3, 3), padding='same', activation='piecewise5')(input_img)
        block1_output = GlobalAveragePooling2D()(conv_1)
        max_pool_1 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(conv_1)
        dropout_1 = Dropout(0.25)(max_pool_1)
        
        conv_2 = Conv2D(128, (3, 3), padding='same', activation='piecewise5')(dropout_1)
        block2_output = GlobalAveragePooling2D()(conv_2)
        max_pool_2 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(conv_2)
        dropout_2 = Dropout(0.01)(max_pool_2)
        
        conv_3 = Conv2D(64, (3, 3), padding='same', activation='piecewise5')(dropout_2)
        block3_output = GlobalAveragePooling2D()(conv_3)
        max_pool_3 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(conv_3)
        dropout_3 = Dropout(0.01)(max_pool_3)
        
        conv_4 = Conv2D(128, (3, 3), padding='same', activation='piecewise5')(dropout_3)
        block4_output = GlobalAveragePooling2D()(conv_4)
        max_pool_4 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(conv_4)
        dropout_4 = Dropout(0.01)(max_pool_4)
        
        conv_5 = Conv2D(64, (3, 3), padding='same', activation='piecewise5')(dropout_4)
        block5_output = GlobalAveragePooling2D()(conv_5)
        max_pool_5 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(conv_5)
        dropout_5 = Dropout(0.01)(max_pool_5)
        
        conv_6 = Conv2D(128, (3, 3), padding='same', activation='piecewise5')(dropout_5)
        block6_output = GlobalAveragePooling2D()(conv_6)
        
        output = keras.layers.concatenate([block1_output, block2_output, 
                                           block3_output, block4_output, 
                                           block5_output, block6_output], 
                                          axis=1)
        # output = Flatten()(output)
        output = Dense(64, activation='piecewise5')(output)
        out = Dense(classes, activation='softmax')(output)
        
        model = Model(inputs=input_img, outputs=out)

    print(model.summary())
    return model


In [ ]:
height = 400
width = 300
channels = 3
classes = 3
ratio_train = 0.8
ratio_val = 0.2
save_path = '/content/drive/My Drive/Colab Notebooks/training_info.xlsx'
pre_trained_model_path = ''
model_save_path = '/content/drive/My Drive/Colab Notebooks/test_model.h5'
data_path = '/content/drive/My Drive/Colab Notebooks/covid19-detection-xray-dataset'
    
if ratio_train + ratio_val > 1:
    print('Train/eval splitting failed')
    exit(0)
X_train, y_train, X_val, y_val = data_processing(data_path,
                                                     height, 
                                                     width,  
                                                     classes)
        
get_custom_objects().update({'piecewise5': Piecewise5(piecewise5)})
input_shape = X_train.shape[1:]

sgd = optimizers.SGD(lr=0.001, momentum=0.9, nesterov=False)
model = custom_network(height, width, channels, classes, pre_trained_model_path)
    
model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer='adam',
                  metrics=['accuracy'])
    
epochs = 25
hist1 = model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_data=(X_val, y_val))
model.save(model_save_path)  # should end with .h5 or .hdf5
    
history = hist1.history
    
score = model.evaluate(X_val, y_val, verbose=0)
score2 = model.evaluate(X_train, y_train, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Training loss:', score2[0])
print('Training accuracy:', score2[1])

ViralPneumonia
COVID-19
Normal
BacterialPneumonia
BacterialPneumonia
COVID-19
Normal
ViralPneumonia
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 400, 300, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 400, 300, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 200, 150, 64) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 200, 150, 64) 0           max_pooling2d_1[0][0]     